In [89]:
import pandas as pd 
import numpy as np 
import seaborn as sns 
import matplotlib.pyplot as plt 
import plotly.express as px 
import plotly.graph_objects as go 
from sklearn.model_selection import TimeSeriesSplit
from sklearn.ensemble import RandomForestRegressor, VotingRegressor
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, accuracy_score, precision_score
from colorama import Fore, Style
from IPython.display import display, HTML 
print('Setup Complete')

Setup Complete


In [90]:
data = pd.read_csv('Apple_News_Analyzed.csv')

data.head(5)

,ticker,Date,title,content,Open,High,Low,Close,Adj Close,Volume,...,stopless_title,stopless_content,title_stemmed,content_stemmed,sentiment_title,sentiment_content,subjectivity_title,subjectivity_content,polarity_title,polarity_content
0,AAPL,7/23/2012,Summer Heat Scorches Europe And U S,Europe flares as summer heat continues Summer...,21.228571,21.639286,20.989643,21.565357,18.257221,487975600.0,...,['summer heat scorches europe and u s '],['europe flares as summer heat continues summ...,['summer heat scorches europe and u s '],['europe flares as summer heat continues summ...,0,1,0.0,0.360324,0.0,0.115828
1,AAPL,7/23/2012,Apple Earnings Preview Quarterly Dip On Deck,Last quarter Apple AAPL reported the 2nd bes...,21.692142,21.774286,21.375357,21.461430,18.169237,565132400.0,...,['apple earnings preview quarterly dip on dec...,['last quarter apple aapl reported the 2nd b...,['apple earnings preview quarterly dip on dec...,['last quarter apple aapl reported the 2nd b...,0,1,0.0,0.415023,0.0,0.099617
2,AAPL,7/23/2012,Trade Apple After Earnings,It may look like a spider web but the mishmas...,20.516430,20.742857,20.357143,20.534643,17.384607,877312800.0,...,['trade apple after earnings'],['it may look like a spider web but the mishm...,['trade apple after earn'],['it may look like a spider web but the mishm...,0,1,0.0,0.350683,0.0,0.083523
3,AAPL,7/24/2012,Market Bait And Switch,That is the sound we are going to hear soon fr...,20.705713,20.728571,20.370001,20.531429,17.381897,406632800.0,...,['market bait and switch'],['that is the sound we are going to hear soon ...,['market bait and switch'],['that is the sound we are going to hear soon ...,0,-1,0.0,0.470486,0.0,-0.003319
4,AAPL,7/27/2012,Will AAPL Fall From The Tree,Apple s AAPL sales for the third quarter mis...,20.536072,20.922501,20.413929,20.898571,17.692715,403936400.0,...,['will aapl fall from the tree '],['apple s aapl sales for the third quarter m...,['will aapl fall from the tree '],['apple s aapl sales for the third quarter m...,0,1,0.0,0.283718,0.0,0.084097


In [91]:
data = data.dropna()
m, n = data.shape

train = data[:1400]
test = data[1400:m]

print('Train shape: ', train.shape)
print('Test shape: ', test.shape)

Train shape:  (1400, 25)
Test shape:  (393, 25)


In [92]:
tss = TimeSeriesSplit(n_splits=200)

predictors = ["Open","High","Low","Close","Volume", "sentiment_title","sentiment_content", "subjectivity_title", "subjectivity_content", "polarity_title", "polarity_content"]

def model(model, predictors):
    for train_index, test_index in tss.split(data):
        train = data.iloc[train_index]
        test = data.iloc[test_index]
        model.fit(train[predictors], train["Close"])

In [93]:
def pred(model, predictors):
    preds = model.predict(test[predictors])
    preds = pd.Series(preds, index=test.index)
    
    return preds

In [94]:
def eval(y_true, y_pred, tolerance=0.05):
    within_tolerance = np.abs((y_true - y_pred) / y_true) <= tolerance
    return np.mean(within_tolerance)

In [95]:
xgb = XGBRegressor()
model(xgb, predictors)
xgb_preds = pred(xgb, predictors)

xgb_mse = mean_squared_error(test['Close'], xgb_preds)
print('MSE in XGBRegressor predictions:\n', xgb_preds)
xgbaccuracy = eval(test["Close"], xgb_preds, tolerance=0.001)
print("Accuracy within 0.1% tolerance:", xgbaccuracy)

MSE in XGBRegressor predictions:
 1400    41.837898
1401    43.218910
1402    43.108292
1403    42.985203
1404    42.749641
          ...    
1788    52.165974
1789    51.633488
1790    52.239891
1791    53.343403
1792    53.183220
Length: 393, dtype: float32
Accuracy within 0.1% tolerance: 0.9440203562340967


In [96]:
lgbm = LGBMRegressor()
model(lgbm, predictors)
lgbm_preds = pred(lgbm, predictors)

lgbm_mse = mean_squared_error(test["Close"], lgbm_preds)
print("MSE in LGBMRegressor predictions: ", lgbm_mse)
lgbmaccuracy = eval(test["Close"], lgbm_preds, tolerance=0.001)
print("Accuracy within 0.1% tolerance:", lgbmaccuracy)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000070 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 503
[LightGBM] [Info] Number of data points in the train set: 193, number of used features: 11
[LightGBM] [Info] Start training from score 19.412289
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, b

In [97]:
trace1 = go.Scatter(x=train.index, y=train["Close"], mode="lines", name="Train")
trace2 = go.Scatter(x=test.index, y=test["Close"], mode="lines", name="Val")
trace5 = go.Scatter(x=test.index, y=xgb_preds, mode="lines", name="XGBoost Preds")
trace6 = go.Scatter(x=test.index, y=lgbm_preds, mode="lines", name="LGMB Preds")

# Combine traces
fig = go.Figure([trace1, trace2, trace5, trace6])

# Update layout
fig.update_layout(
    title="Model Predictions",
    xaxis=dict(title="Date", tickfont=dict(size=14)),
    yaxis=dict(title="Close Price USD ($)", tickfont=dict(size=14)),
    legend=dict(x=0.02, y=0.98),
    plot_bgcolor="black",
    paper_bgcolor="black",
    font=dict(color="white"),
)

# Show plot
fig.show()